# SAA-Aggression Dataset Inference

In [1]:
"""
Dataset      : Saag01
Epochs       : 80 (total)
Iterations   : 1361 (train), 341 (val), 190 (test)
Batch_size   : 48
Iters/Epoch  : 28

Arch         : BNInception
Modality     : ARP
""";

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
def extract_index(basename):
    index = basename.split('_')[-1]
    return index

# list all files' abspath in a directory
def abs_listdir(directory):
    directory = os.path.abspath(directory)
    return [
        os.path.join(directory, filename)
        for filename in os.listdir(directory)
    ]

# format indexing
def format_files_indexing(file_paths):
    formatted_filepaths = []
    for file_path in file_paths:
        # assuming digit is located at the end of file name
        basename, ext = os.path.splitext(file_path)
        index = extract_index(basename)
        if index.isdigit():
            formatted_filename = basename.replace(index, "{:05d}".format(int(index))) + ext

            os.rename(file_path, formatted_filename)
            formatted_filepaths.append(formatted_filename)

    return sorted(formatted_filepaths)

In [4]:
SCORES_DIR = 'Scores/'

SCORES_MAPDIR = {
    'SAAG01':{
        'ARP': {
            '3_segments': os.path.join('Scores/', 'saag01_bni_arp_seg_3_scores_split_0'),
            '7_segments': os.path.join('Scores/', 'saag01_bni_arp_seg_7_scores_split_0')
        },
        'Flow': {
            '3_segments': os.path.join('Scores/', 'saag01_bni_flow_seg_3_scores_split_0'),
            '7_segments': ''
        }
    }
}

## Dense Optical Flow (type I)

In [8]:
SAAG01_Flow_3seg_dir = SCORES_MAPDIR['SAAG01']['Flow']['3_segments']  # choose the one with 3 segments

In [9]:
# Index formatting for scores' files
# this is for sorting consistency
saag01_flow_3seg_files = format_files_indexing(abs_listdir(SAAG01_Flow_3seg_dir))

saag01_flow_3seg_files[10:15]

['/home/sliu/Desktop/inference/Scores/saag01_bni_flow_seg_3_scores_split_0/saag01_bni_flow_seg_3_scores_split_0_00308.npy',
 '/home/sliu/Desktop/inference/Scores/saag01_bni_flow_seg_3_scores_split_0/saag01_bni_flow_seg_3_scores_split_0_00336.npy',
 '/home/sliu/Desktop/inference/Scores/saag01_bni_flow_seg_3_scores_split_0/saag01_bni_flow_seg_3_scores_split_0_00364.npy',
 '/home/sliu/Desktop/inference/Scores/saag01_bni_flow_seg_3_scores_split_0/saag01_bni_flow_seg_3_scores_split_0_00392.npy',
 '/home/sliu/Desktop/inference/Scores/saag01_bni_flow_seg_3_scores_split_0/saag01_bni_flow_seg_3_scores_split_0_00420.npy']

In [10]:
# Load labels
split_val_0_file = 'saag01_validation_split_0.txt'
label_df = pd.read_csv(split_val_0_file, 
                       delim_whitespace=True,
                       header=None,
                       names=['path', 'n_frames', 'label']
                       
# drop last 5 items since we used droplast=True, 
# hence batchsize = 48, and valset = 341; then the remainder is 5           )
labels = label_df['label'][:-5] 

labels.shape

(336,)

In [13]:
# Try one numpy score from a file
saag01_score_sample = np.load(saag01_flow_3seg_files[-1])

pd.DataFrame(saag01_score_sample).head()  # logits for two scores (before softmax)

,0,1
0,-0.712530,0.718903
1,-1.565154,1.555244
2,2.895567,-2.874315
3,1.898230,-1.891159
4,-3.237809,3.185252


In [14]:
saag01_score_sample.shape

(336, 2)

### One-stream evaluation

#### 1. Validation result

In [16]:
# Num_segments: 3
for scorefile in saag01_flow_3seg_files:
    itername, _ = os.path.splitext(scorefile)
    itername = extract_index(itername)
    
    print("Iter: %s" % itername)
    scores = np.load(scorefile)
    pred1 = np.argmax(scores, axis=1)
    acc = np.mean(pred1 == labels)
    print("Flow Acc: %f" % acc)

Iter: 00028
Flow Acc: 0.717262
Iter: 00056
Flow Acc: 0.889881
Iter: 00084
Flow Acc: 0.830357
Iter: 00112
Flow Acc: 0.883929
Iter: 00140
Flow Acc: 0.895833
Iter: 00168
Flow Acc: 0.863095
Iter: 00196
Flow Acc: 0.845238
Iter: 00224
Flow Acc: 0.866071
Iter: 00252
Flow Acc: 0.824405
Iter: 00280
Flow Acc: 0.907738
Iter: 00308
Flow Acc: 0.898810
Iter: 00336
Flow Acc: 0.910714
Iter: 00364
Flow Acc: 0.910714
Iter: 00392
Flow Acc: 0.875000
Iter: 00420
Flow Acc: 0.931548
Iter: 00448
Flow Acc: 0.922619
Iter: 00476
Flow Acc: 0.901786
Iter: 00504
Flow Acc: 0.872024
Iter: 00532
Flow Acc: 0.845238
Iter: 00560
Flow Acc: 0.889881
Iter: 00588
Flow Acc: 0.901786
Iter: 00616
Flow Acc: 0.910714
Iter: 00644
Flow Acc: 0.916667
Iter: 00672
Flow Acc: 0.919643
Iter: 00700
Flow Acc: 0.910714
Iter: 00728
Flow Acc: 0.910714
Iter: 00756
Flow Acc: 0.922619
Iter: 00784
Flow Acc: 0.925595
Iter: 00812
Flow Acc: 0.922619
Iter: 00840
Flow Acc: 0.919643
Iter: 00868
Flow Acc: 0.916667
Iter: 00896
Flow Acc: 0.919643
Iter: 00

#### 2. Test Result

In [17]:
# load test labels
split_test_file = 'saag01_test_split.txt'
test_labels_df = pd.read_csv(split_test_file, 
                       delim_whitespace=True,
                       header=None,
                       names=['path', 'n_frames', 'label']
                      )
test_labels = test_labels_df['label']

In [18]:
# Num_segments: 3
test_score_file = os.path.join(SAAG01_Flow_3seg_dir,
                               'saag01_bni_flow_seg_3_test_scores.npy')

In [19]:
test_labels.shape

(190,)

In [20]:
# Num_segments: 3
print("Test iterations")
scores = np.load(test_score_file)
pred1 = np.argmax(scores, axis=1)
acc = np.mean(pred1 == test_labels)
print("Flow Test Acc: %f" % acc)

Test iterations
Flow Test Acc: 0.900000
